In [1]:
%matplotlib inline
import requests
import json
import time
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

In [2]:
# Hàm thu thập dữ liệu từ một khoảng thời gian cụ thể
def collect_data(api_url):
    weather_data = []
    success = False
    while not success:
        print(f"GET: {api_url}")
        r = requests.get(api_url)
        if r.ok:
            # lấy kết quả json
            pydata = json.loads(r.text)
            weather_data.extend(pydata["days"])  # Lưu dữ liệu từ 'days'
            print("Successful! Remaining cost:", pydata.get('remainingCost', 'Unknown'))
            print()
            success = True
        else:
            print("Fail! Try again. Status code:", r.status_code)
            time.sleep(5)
        
    return weather_data


In [3]:
# Hàm tạo URL API từ các tham số đầu vào
def create_api_url(location, start_date, end_date, api_key, unit_group='metric', include='days'):
    return f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date}/{end_date}?unitGroup={unit_group}&include={include}&key={api_key}&contentType=json'

# Hàm thu thập dữ liệu trong khoảng thời gian từ nhiều năm
def collect_all_data(start_date, end_date, location, api_key):
    # Tạo URL API
    api_url = create_api_url(location, start_date, end_date, api_key)
    # Thu thập dữ liệu thời tiết
    weather_data = collect_data(api_url)
    
    return weather_data

In [4]:
# API key (có thể thay đổi)
api_key1 = 'TE7HPVTQ4Q4B7LWSLDSU38BRF'
api_key2 = "VNTCFNZ5VLGZC3FW4F85U8425"
# Thông số đầu vào (có thể thay đổi)
location = 'hanoi'
start_date = '2022-09-01'
end_date = '2024-09-01'

# # Thu thập dữ liệu
# all_weather_data = collect_all_data(start_date, end_date, location, api_key2)

# # In số lượng dữ liệu thu thập được
# print('Number of data:', len(all_weather_data))

GET: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/hanoi/2022-09-01/2024-09-01?unitGroup=metric&include=days&key=VNTCFNZ5VLGZC3FW4F85U8425&contentType=json
Successful! Remaining cost: Unknown

Number of data: 732


In [5]:
# # Lưu dữ liệu vào DataFrame và xuất ra CSV
# data_df = pd.DataFrame(all_weather_data)
# data_df.rename(lambda name: name[0].upper() + name[1:], axis='columns', inplace=True)
# data_df.to_csv('historical_weather_data.csv', index=False)

# # Hiển thị 5 dòng đầu tiên của dữ liệu
# data_df.head()

,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Feelslikemax,Feelslikemin,Feelslike,Dew,Humidity,...,Sunrise,SunriseEpoch,Sunset,SunsetEpoch,Moonphase,Conditions,Description,Icon,Stations,Source
0,2022-09-01,1661965200,31.7,25.0,28.6,38.4,25.0,34.0,25.5,83.4,...,05:40:07,1661985607,18:13:02,1662030782,0.17,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"[48820099999, 48823099999, 48825099999, 488310...",obs
1,2022-09-02,1662051600,34.5,26.0,30.2,41.2,26.0,35.1,24.4,72.6,...,05:40:23,1662072023,18:12:08,1662117128,0.21,"Rain, Partially cloudy",Partly cloudy throughout the day with early mo...,rain,"[48820099999, 48823099999, 48825099999, 488310...",obs
2,2022-09-03,1662138000,34.7,27.0,30.8,44.2,31.2,37.6,25.4,73.9,...,05:40:38,1662158438,18:11:14,1662203474,0.24,"Rain, Partially cloudy",Partly cloudy throughout the day with morning ...,rain,"[48820099999, 48823099999, 48825099999, 488310...",obs
3,2022-09-04,1662224400,30.5,25.0,27.6,35.9,25.0,30.6,23.9,80.7,...,05:40:53,1662244853,18:10:20,1662289820,0.25,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"[48820099999, 48823099999, 48825099999, 488310...",obs
4,2022-09-05,1662310800,34.2,26.0,29.4,37.3,26.0,31.6,22.3,67.7,...,05:41:08,1662331268,18:09:25,1662376165,0.31,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,"[48820099999, 48823099999, 48825099999, 488310...",obs


In [6]:
# Đọc dữ liệu từ file CSV
csv_file_path = 'historical_weather_data.csv'  # Đường dẫn đến file CSV
all_weather_data = pd.read_csv(csv_file_path)

# In số lượng dữ liệu đã tải về
print('Number of data loaded from CSV:', len(all_weather_data))


Number of data loaded from CSV: 732


In [7]:
# Kiểm tra số lượng giá trị không bị thiếu trong mỗi cột
non_null_counts = all_weather_data.count()
print("Non-null counts per column:")
print(non_null_counts)

# Tính tỷ lệ phần trăm giá trị thiếu trong mỗi cột
missing_percentage = all_weather_data.isnull().mean() * 100
print("\nMissing percentage per column:")
print(missing_percentage)

# Lọc ra các cột có tỷ lệ dữ liệu thiếu lớn hơn 20%
missing_columns = missing_percentage[missing_percentage > 20]
print("\nColumns with more than 20% missing data:")
print(missing_columns)

# Tìm các cột có ít hơn 50 giá trị không bị thiếu
low_data_columns = non_null_counts[non_null_counts < 50]
print("\nColumns with less than 50 non-null values:")
print(low_data_columns)

Non-null counts per column:
Datetime          732
DatetimeEpoch     732
Tempmax           732
Tempmin           732
Temp              732
Feelslikemax      732
Feelslikemin      732
Feelslike         732
Dew               732
Humidity          732
Precip            732
Precipprob        732
Precipcover       732
Preciptype        544
Snow              732
Snowdepth         732
Windgust          732
Windspeed         732
Winddir           732
Pressure          732
Cloudcover        732
Visibility        732
Solarradiation    732
Solarenergy       732
Uvindex           732
Severerisk        732
Sunrise           732
SunriseEpoch      732
Sunset            732
SunsetEpoch       732
Moonphase         732
Conditions        732
Description       732
Icon              732
Stations          732
Source            732
dtype: int64

Missing percentage per column:
Datetime           0.00000
DatetimeEpoch      0.00000
Tempmax            0.00000
Tempmin            0.00000
Temp               0.00000
